# <font color='navy'> Nurse Scheduling Problem
## <font color='navy'> Pyomo Approach

### <font color='navy'>Libraries

In [1]:
# !pip install pyomo
# !pip install gurobipy

from __future__ import division
from pyomo.environ import*
from pyomo.opt import*
import numpy as np

### <font color='navy'>Problem proposal

#### <font color='navy'> Variables
- Nurses: $n \in N = \{1...5\}$
- Shifts: $s \in S = \{1...3\}$
- Days: $d \in D = \{1, 7\}$

#### <font color='navy'>Objective function and constraints:

- Each shift is assigned to a single nurse per day:
$$\sum^{}_{n \in N}\, X_{nsd} = 1\;\;\;\;\;\;\;\;\;\;\;\;(1)$$
    
- Each nurse must work at most one shift per day:
$$\sum^{}_{s \in S}\, X_{nsd} \leq 1\;\;\;\;\;\;\;\;\;\;\;\;(2)$$

- Each nurse works at least 2 shifts per week:
$$\sum^{}_{d \in D}\,\sum^{}_{s \in S}\, X_{nsd} \geq 2\;\;\;\;\;\;\;\;\;\;\;\;(3)$$

- Each nurse must not work consecutive shifts:
$$X_{i,3,t} + X_{i,1,t+1} \leq 1\;\;\;\;\;\;\;\;\;\;\;\;(4)$$

- **Objective function:** minimize the total number of weekly shifts worked by each nurse.

$$Minimize \sum_{n,s,d}{X_{n,s,d}}$$

In [4]:
model = AbstractModel()
model.n = RangeSet(1,5) # Individuals (i)
model.s = RangeSet(1,3) # Shifts (j)
model.d = RangeSet(1,7) # Days (t)

model.X = Var(model.n, model.s, model.d, within=Binary)
model.Nshift= Var(within=NonNegativeReals)

# There must be at least 1 nurse on each shift:
def Constraint_1(model, d, s):
    return sum(model.X[n,s,d] for n in model.n) == 1
model.C1 = Constraint(model.d, model.s, rule=Constraint_1)

# Every nurse must not work more than 1 shift per day:
def Constraint_2(model, d, n):
    return sum(model.X[n,s,d] for s in model.s) <= 1
model.C2 = Constraint(model.d, model.n, rule=Constraint_2)

# Every nurse must work at least 2 shifts per week:
def Constraint_3(model, d, n):
    return sum(model.X[n,s,d] for s in model.s for d in model.d) >= 2
model.C3 = Constraint(model.d, model.n, rule=Constraint_3)

# Each individual must not work consecutive shifts:
def Constraint_4(model, d, n):
    if d<7:
        return model.X[n,3,d] + model.X[n,3,d+1] <= 1
    else:
        return Constraint.Skip
model.C4 = Constraint(model.d,model.n, rule=Constraint_4)

# Every individual must get at least one night shift break, preferably two.
def Constraint_5(model,d,n):
    if d<7:
        return model.X[n,3,d]+ model.X[n,1,d+1] <= 1
    else:
        return Constraint.Skip
model.C5 = Constraint(model.d, model.n, rule=Constraint_5)

def Constraint_6(model, n):
    return sum(model.X[n,3,d] for d in model.d) <= model.Nshift
model.C6 = Constraint(model.n,  rule=Constraint_6)

def Obj_Function(model):
    #return quicksum(model.X[i,j,t] for i in model.i for j in model.j for t in model.t )
    return sum(model.X[n,s,d] for n in model.n for s in model.s for d in model.d) 
model.obj = Objective(rule=Obj_Function, sense=minimize)

solver = SolverFactory('gurobi')
instance = model.create_instance()
result = solver.solve(instance, load_solutions=True) # solves and updates instance

In [5]:
for d in model.d:
    print('Day {}'.format(d))
    for n in model.n:
        for s in model.s:
            if value(instance.X[n,s,d]) == 1:
                print('  Nurse {} works at shift {}'.format(n, s))
#             else:
#                 print('Nurse {} is not requested'.format(n))

Day 1
  Nurse 1 works at shift 1
  Nurse 2 works at shift 2
  Nurse 5 works at shift 3
Day 2
  Nurse 1 works at shift 3
  Nurse 2 works at shift 2
  Nurse 3 works at shift 1
Day 3
  Nurse 2 works at shift 2
  Nurse 4 works at shift 1
  Nurse 5 works at shift 3
Day 4
  Nurse 1 works at shift 1
  Nurse 2 works at shift 3
  Nurse 5 works at shift 2
Day 5
  Nurse 1 works at shift 2
  Nurse 3 works at shift 3
  Nurse 5 works at shift 1
Day 6
  Nurse 1 works at shift 2
  Nurse 2 works at shift 1
  Nurse 4 works at shift 3
Day 7
  Nurse 1 works at shift 3
  Nurse 2 works at shift 1
  Nurse 3 works at shift 2
